In [2]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from utils import QuantumAdder
from quantum_subtractor import QuantumSubtractor
from grover_max import GroverMax
from grover_comparator import GroverComparator
import matplotlib.pyplot as plt


class QuantumSystolicArray:
    def __init__(self, seq_a, seq_b, num_qubits, match_score=2, gap_penalty=-1, mismatch_score=-1):
        self.seq_a = seq_a
        self.seq_b = seq_b
        self.num_qubits = num_qubits
        self.match_score = match_score
        self.gap_penalty = gap_penalty
        self.mismatch_score = mismatch_score
        self.global_max_score = -200
        self.global_max_position = None
        self.array = self.build_array()
        self.alignment_a = []
        self.alignment_b = []
        self.diagonals = self.compute_diagonals()

    def compute_diagonals(self):
        """
        Compute all anti-diagonal coordinates for the systolic array.
        Each diagonal can be processed in parallel.
        """
        """
        Entire matrix is filled in a diagonal manner, each cell is assigned with the value of diagonal
        	T	    G	    C	    A
        A	(0,0)	(0,1)	(0,2)	(0,3)
        C	(1,0)	(1,1)	(1,2)	(1,3)
        G	(2,0)	(2,1)	(2,2)	(2,3)
        T	(3,0)	(3,1)	(3,2)	(3,3)
        Each cell will hold its cell number not the value of the cell
        """
        diagonals = []
        max_len = len(self.seq_a) + len(self.seq_b) - 1
        for diag in range(max_len):
            diagonal = []
            for i in range(len(self.seq_a)):
                j = diag - i
                if 0 <= j < len(self.seq_b):
                    diagonal.append((i, j))
            if diagonal:  # Only add non-empty diagonals
                diagonals.append(diagonal)
        return diagonals

    def build_array(self):
        """
        Constructs the 2D systolic array using QuantumCell instances.
        """
        qarray = []
        for i in range(len(self.seq_a)):
            qarray.append([])
            for j in range(len(self.seq_b)):
                qarray[i].append(QuantumCell(self.num_qubits, self.seq_a[i], self.seq_b[j]))
        return qarray

    def process_diagonal(self, diagonal):
        """
        Process a single diagonal of cells in parallel.
        
        Args:
            diagonal (list): List of (i,j) coordinates for cells in this diagonal
        """
        results = []
        # These cells can be processed in parallel
        for i, j in diagonal:
            # Get scores from neighbors (already processed in previous diagonals)
            diag_score = self.get_score(i-1, j-1) if i > 0 and j > 0 else 0
            up_score = self.get_score(i-1, j) if i > 0 else 0
            left_score = self.get_score(i, j-1) if j > 0 else 0
            
            # Calculate cell score
            cell = self.array[i][j]
            #entire smith waterman operation is executed in the following function
            score = cell.calculate_score(diag_score, up_score, left_score, 
                                      self.match_score, self.gap_penalty, self.mismatch_score)
            
            # Update global maximum if necessary
            if score > self.global_max_score:
                self.global_max_score = score
                self.global_max_position = (i, j)
            
            results.append((i, j, score))
        return results

    def process_array(self):
        """
        Process the entire systolic array diagonal by diagonal.
        Each diagonal's cells can be processed in parallel.
        """
        #print("self.diagonals:",self.diagonals)
        for diagonal in self.diagonals:
            # Process all cells in this diagonal in parallel
            diagonal_results = self.process_diagonal(diagonal)
            
            # Optional: collect and process results if needed
            for i, j, score in diagonal_results:
                if score > self.global_max_score:
                    self.global_max_score = score
                    self.global_max_position = (i, j)

    def get_score(self, i, j):
        """
        Safely get score from array position.
        """
        if i < 0 or j < 0:
            return 0
        return self.array[i][j].score

    def traceback_alignment(self):
        """
        Trace back from the maximum score to find the alignment.
        """
        if not self.global_max_position:
            return "", ""
        
        i, j = self.global_max_position
        align_a = []
        align_b = []
        
        #based on cell.direction which was updated in previous smith waterman function traceback happens,
        #if direction is diag then both the cells are minus by 1 
        #if up or left based on that i or j is minus by 1
        #the result alignment is shared
        while i >= 0 and j >= 0 and self.array[i][j].score > 0:
            cell = self.array[i][j]
            if cell.direction == 'diag':
                align_a.append(self.seq_a[i])
                align_b.append(self.seq_b[j])
                i -= 1
                j -= 1
            elif cell.direction == 'up':
                align_a.append(self.seq_a[i])
                align_b.append('-')
                i -= 1
            else:  # left
                align_a.append('-')
                align_b.append(self.seq_b[j])
                j -= 1
                
        self.alignment_a = ''.join(reversed(align_a))
        self.alignment_b = ''.join(reversed(align_b))
        return self.alignment_a, self.alignment_b


class QuantumCell:
    def __init__(self, num_qubits, a, b):
        self.num_qubits = num_qubits
        self.a = a
        self.b = b
        self.simulator = StatevectorSampler()
        self.score = 0
        self.direction = None
        self.is_local_max = False
        self.processed = False  # Flag to track if cell has been processed

    def match(self):
        """
        Quantum circuit for nucleotide matching.
        """
        qc = QuantumCircuit(5, 1)
        
        # Encode sequence A nucleotides
        if self.a == 'C':
            qc.x(1)
        elif self.a == 'T':
            qc.x(0)
            qc.x(1)
        elif self.a == 'G':
            qc.x(0)

        # Encode sequence B nucleotides
        if self.b == 'C':
            qc.x(3)
        elif self.b == 'T':
            qc.x(2)
            qc.x(3)
        elif self.b == 'G':
            qc.x(2)

        # Quantum comparison operations
        qc.cx(0, 2)
        qc.cx(1, 3)
        qc.x(2)
        qc.x(3)
        qc.ccx(2, 3, 4)
        qc.measure(4, 0)
        #display(qc.draw(output='mpl'))
        # Draw the circuit
        #qc.draw("mpl")
        #print(qc.draw())
        #plt.show()
        #

        return qc

    def run_cell(self):
        """
        Run the quantum circuit for the cell and return the match result.
        """
        qc = self.match()
        result = self.simulator.run([qc]).result()[0].data.c.get_bitstrings()[0]
        return result == '1'

    def calculate_score(self, diag_score, up_score, left_score, match_score, gap_penalty, mismatch_score):
        if self.processed:
            return self.score
            
        # Determine if there's a match using quantum circuit
        is_match = self.run_cell() #Before this line only the matrix part was initiated only in run_cell function quantum circuit is initiated 
                                   # and comparision between two sequence in done 
        # Test
        diag_comparator = GroverComparator(diag_score, abs(mismatch_score))
        up_comparator = GroverComparator(up_score, gap_penalty)
        left_comparator = GroverComparator(left_score, gap_penalty)
        
        #1. If the value matches then quantum addition is performed with the diagonal score and match_score -2
        #2. else checks if above diagonal number is greater than or euql if yes then quantum subtract operation is performed with mismatch score
        #3. else quantum subtract operations is performed in reversed manner 
        if is_match:
            match_mismatch_score = self._quantum_add(diag_score, match_score)
        elif diag_comparator.check_larger_or_equal(): #True if number_1 ≥ number_2 then executes inside this block, if not moves to elase
            match_mismatch_score = self._quantum_subtract(diag_score, abs(mismatch_score))
        else:
            match_mismatch_score = -self._quantum_subtract( abs(mismatch_score), diag_score)

        #1. If up number is greater than or euql if yes then quantum subtract operation is performed with gap penalty score
        #2. else gap_up is filled as 0
        if up_comparator.check_larger_or_equal():
            gap_up = self._quantum_subtract(up_score, abs(gap_penalty))
        else:
            gap_up = 0
        
        #1. If left number is greater than or equal if yes then quantum subtract operation is performed with gap penalty score
        #2. else gap_left is filled as 0
        if left_comparator.check_larger_or_equal():
            gap_left = self._quantum_subtract(left_score, abs(gap_penalty)) 
        else:
            gap_left = 0
        
        # As per smith watermann algorithm grover search compares the diagnoal, up and left score if they are in negative then 0 will be the highest score
        # and this score is updated in self.score(self.score hold the highest score)
        grover_search = GroverMax(0, int(match_mismatch_score), int(gap_up), int(gap_left))
        self.score = grover_search.find_max()
        
        if self.score == match_mismatch_score:
            self.direction = 'diag'
        elif self.score == gap_up:
            self.direction = 'up'
        elif self.score == gap_left:
            self.direction = 'left'
        else:  
            self.direction = None
            
        self.processed = True
        return self.score


    def _quantum_add(self, a_val, b_val):
        """Perform quantum addition."""
        if a_val == float('-inf') or b_val == float('-inf'):
            return -20
        qadd = QuantumAdder(self.num_qubits, 40)
        return qadd.run_addition(a_val, b_val) # This function calls utils.py file,
        #1. in the file values are convereted to binary
        #2. Classic and quantum registers are created
        #3. binary equivalent simulation is executed in the qubits 
        #4. The above executed qubits are recorded in register, then transformed using QFT (i.e) addition is performed using QFT operation

    def _quantum_subtract(self, a_val, b_val):
        """Perform quantum subtraction."""
        if a_val == float('-inf') or b_val == float('-inf'):
            return -20
        comp = GroverComparator(a_val, b_val)
        if comp.check_larger_or_equal():
            qsub = QuantumSubtractor(self.num_qubits)
            return qsub.run_subtract(a_val, b_val)
        return -20


if __name__ == "__main__":
    # Example usage
    seq_a = "ACACACTA"
    seq_b = "AGCACACA"
    num_qubits = 5
    
    # Initialize and run alignment
    aligner = QuantumSystolicArray(seq_a, seq_b, num_qubits)
    aligner.process_array()
    alignment_a, alignment_b = aligner.traceback_alignment()
    
    # Print results
    print(f"Sequence A: {seq_a}")
    print(f"Sequence B: {seq_b}")
    print(f"Alignment A: {alignment_a}")
    print(f"Alignment B: {alignment_b}")
    print(f"Maximum Score: {aligner.global_max_score}")
    
    # Draw and display the quantum circuit
    #circuit_image = QuantumCell.draw("mpl")
    #plt.show(block=True)
    
  

Sequence A: ACACACTA
Sequence B: AGCACACA
Alignment A: A-CACACTA
Alignment B: AGCACAC-A
Maximum Score: 12
